In [1]:
import pandas as pd
data = pd.read_excel("Final_Scores_ISCO08_Gmyrek_et_al_2025.xlsx")
data.head(3)

,label4d,label1d,ISCO_08,Title,taskID,Task_ISCO,score_2023,Weaviate Status,predicted_score_2025_gpt4o,prediction_justification_gpt4o,...,predicted_score_2025_gemini,prediction_justification_gemini,score_2025,source,mean_score_2023,mean_score_2025,SD_2023,SD_2025,potential25,potential23
0,1111 - Legislators,1 - Managers,1111,Legislators,1,Presiding over or participating in the proceed...,0.1,success,0.330,The task of presiding over or participating in...,...,0.35,The task of presiding over legislative proceed...,0.3400,predicted,0.22,0.31,0.07,0.04,Not Exposed,Not affected
1,1111 - Legislators,1 - Managers,1111,Legislators,2,"Determining, formulating and directing policie...",0.3,success,0.365,"The task of determining, formulating, and dire...",...,0.35,"The task of ""Determining, formulating and dire...",0.3575,predicted,0.22,0.31,0.07,0.04,Not Exposed,Not affected
2,1111 - Legislators,1 - Managers,1111,Legislators,3,"Making, ratifying, amending or repealing laws,...",0.2,success,0.330,"The task of making, ratifying, amending, or re...",...,0.25,"The task of making, ratifying, amending, or re...",0.2900,predicted,0.22,0.31,0.07,0.04,Not Exposed,Not affected


In [2]:
import pandas as pd
import json

# Load the data
data = pd.read_excel("4digits_with_tasks.xlsx")

# Function to build the hierarchical structure
def build_hierarchy(df):
    hierarchy = {"name": "ISCO-08", "children": []}

    # Group data by hierarchical levels
    grouped = df.groupby(["isco08_1d", "isco08_2d", "isco08_3d", "isco08_4d"])

    for (isco08_1d, isco08_2d, isco08_3d, isco08_4d), group in grouped:
        # Find or create the first-level node
        level1 = next((x for x in hierarchy["children"] if x["name"] == isco08_1d), None)
        if not level1:
            level1 = {"name": isco08_1d, "children": []}
            hierarchy["children"].append(level1)

        # Find or create the second-level node
        level2 = next((x for x in level1["children"] if x["name"] == isco08_2d), None)
        if not level2:
            level2 = {"name": isco08_2d, "children": []}
            level1["children"].append(level2)

        # Find or create the third-level node
        level3 = next((x for x in level2["children"] if x["name"] == isco08_3d), None)
        if not level3:
            level3 = {"name": isco08_3d, "children": []}
            level2["children"].append(level3)

        # Find or create the fourth-level node
        level4 = next((x for x in level3["children"] if x["name"] == isco08_4d), None)
        if not level4:
            risk_value = group["potential25"].iloc[0]  # Assign risk color from first occurrence
            level4 = {"name": isco08_4d, "risk": risk_value, "children": []}
            level3["children"].append(level4)

        # Add fifth-level tasks with task colors
        for _, row in group.iterrows():
            level4["children"].append({"name": row["isco08_5d"], "risk": row["task_color"]})

    return hierarchy

# Build hierarchy from data
hierarchy_data = build_hierarchy(data)

# Save to JSON file
output_file = "output_data.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(hierarchy_data, f, ensure_ascii=False, indent=2)

print(f"JSON file saved as {output_file}")


JSON file saved as output_data.json
